OBTAINING ALL TWEETS FROM A USER

In [ ]:
import requests
import json
import pandas as pd
import tweepy

In [ ]:
#pip install --upgrade tweepy  # needed v.4

In [ ]:
# LA LLAVE
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAF02cQEAAAAAAw4Ko%2BlvaP%2FEKo4jetBjHN%2BGEpY%3D8ot7Zeg7DytI7xLRXuZO23cSWSLBIAHPMKrw9b8jebi9j7nWsI"

#Extracción de data a partir del @username.
Basado en Tweepy v4.9.0

In [ ]:
input_user = 'mferna'

In [ ]:
def extract_tweet_data(input_user):
    """
    Extracts data from a Twitter account.

    Parameters:

    Returns:
    pd.DataFrame
        A dataframe of the user tweets containing the language, time of creation,
        text, tweet id, number of retweets and number of likes.
    """
    # Authorize access
    client = tweepy.Client(BEARER_TOKEN)

    # Get user ID from username
    user_id = client.get_user(username=input_user).data.id

    # Get Tweets timeline (check all pages, max 100 tweets per page, max 32 pages)
    df_list = []
    df_len0 = []
    df_len = []
    for response in tweepy.Paginator(client.get_users_tweets, user_id,
                                    tweet_fields='created_at,lang,public_metrics',
                                    max_results=100, limit=32):
        page_df = pd.DataFrame.from_dict(response.data)
        df_len0.append(len(page_df))
        # Delete non-spanish tweets
        page_df = page_df.drop(page_df[page_df.lang !='es'].index)
        page_df = page_df.drop(['lang'], axis=1)
        df_len.append(len(page_df))
        # Organize columns
        retweet_count = []
        like_count = []
        for item in page_df['public_metrics'].tolist():
          retweet_count.append(item['retweet_count'])
          like_count.append(item['like_count'])
        page_df = page_df.drop(['public_metrics'], axis=1)
        page_df['retweet_count'] = retweet_count
        page_df['like_count'] = like_count
        df_list.append(page_df)
    es_tweets = round(sum(df_len)*100/sum(df_len0),1)
    print(f'Obtained {es_tweets}% tweets in spanish from total of {sum(df_len0)} tweets from user @{input_user}.')
    tweets_df = pd.concat(df_list, ignore_index=True)
    return tweets_df

OLD EXTRACTION CODE (only works for obtaining user info, and only 100 tweets max)\
Es una modificación de los ejemplos para obtener [info del user](https://github.com/twitterdev/Twitter-API-v2-sample-code/blob/main/User-Lookup/get_users_with_bearer_token.py) y [sus tweets](https://github.com/twitterdev/Twitter-API-v2-sample-code/blob/main/User-Tweet-Timeline/user_tweets.py)


In [ ]:
def create_url(username, twitter_object):
    if twitter_object == 'user':
      usernames = f'usernames={input_user}'
      user_fields = 'user.fields=description,created_at,protected,public_metrics,verified'
      url = f'https://api.twitter.com/2/users/by?{usernames}&{user_fields}'
    elif twitter_object == 'tweet':
      user_dict = get_json('user')
      user_id = user_dict['data'][0]['id']
      url = f'https://api.twitter.com/2/users/{user_id}/tweets?max_results=100'
    else:
      print('Error. Wrong Twitter Object.')
      return None
    return url


def get_params(twitter_object):
    if twitter_object == 'tweet':
      params = {"tweet.fields": "created_at,lang,public_metrics"}
    else:
      print('Error. Wrong Twitter Object.')
      return None
    return params


def bearer_oauth_user(r):
    r.headers["Authorization"] = f"Bearer {BEARER_TOKEN}"
    r.headers["User-Agent"] = "v2UserLookupPython"
    return r


def bearer_oauth_tweet(r):
    r.headers["Authorization"] = f"Bearer {BEARER_TOKEN}"
    r.headers["User-Agent"] = "v2UserTweetsPython"
    return r


def connect_to_endpoint(url, twitter_object):
    if twitter_object == 'user':
      response = requests.request("GET", url, auth=bearer_oauth_user,)
    elif twitter_object == 'tweet':
      params = get_params('tweet')
      response = requests.request("GET", url, auth=bearer_oauth_tweet,params=params)
    else:
        print('Error. Wrong Twitter Object.')
        return None
    if response.status_code != 200:
        raise Exception(f'Request returned an error: {response.status_code} {response.text}')
    return response.json()


def get_json(twitter_object):
    if twitter_object == 'user':
      url = create_url(input_user, 'user')
      json_response = connect_to_endpoint(url, 'user')
    elif twitter_object == 'tweet':
      url = create_url(input_user, 'tweet')
      json_response = connect_to_endpoint(url, 'tweet')
    else:
        print('Error. Wrong Twitter Object.')
        return None
    return json_response


def extract_user_data(username):  # MAIN FILE
    """
    Extracts data from a Twitter account.

    Parameters:

    Returns:
    dict
        A dict containing the releveant variables about the user
    """
    # Get user ID from username
    user_dict = get_json('user')
    #user_id = user_dict['data'][0]['id']

    # Get Tweets timeline
    #tweet_dict = get_json('tweet')
    #tweet_pd = pd.DataFrame.from_dict(tweet_dict['data'])
    return user_dict

#OUTPUTS:

In [ ]:
extract_user_data(input_user)  ## Dict with user info

{'data': [{'created_at': '2008-04-02T01:08:53.000Z',
   'description': 'Médico-Epidemíologo, Doctor en Educación, Profesor Titular,Coordinador de Investigaciòn Facultad Med UCV. hablamos de TIC en Investigación, Educación y Salud.',
   'id': '14279589',
   'name': 'Mariano Fernández S',
   'protected': False,
   'public_metrics': {'followers_count': 6093,
    'following_count': 2969,
    'listed_count': 300,
    'tweet_count': 58147},
   'username': 'mferna',
   'verified': False}]}

In [ ]:
extract_tweet_data(input_user)  ## DataFrame with all tweets in spanish from user

Obtained 73.3% tweets in spanish from total of 3198 tweets from user @mferna.


,created_at,id,text,retweet_count,like_count
0,2022-05-03 01:37:08+00:00,1521302809450369026,Guía de evaluación para la conservación de dat...,2,1
1,2022-04-27 15:56:31+00:00,1519344752981909504,RT @gorka_orive: Os recomiendo seguir la infor...,115,0
2,2022-04-27 15:56:09+00:00,1519344657590800391,RT @DrPontecarlosi: Nuevas guías sobre uso de ...,74,0
3,2022-04-27 14:56:02+00:00,1519329529474555905,Sobre la Hepatitis Infantil Desconocida... htt...,0,0
4,2022-04-26 01:20:00+00:00,1518761781325451266,"Adenovirus 41 o covid, sospechosos tras las he...",1,0
...,...,...,...,...,...
2339,2019-10-02 14:51:45+00:00,1179408633756618754,RT @campussanofi: 👍👏 @AbbottGlobal y @sanofi s...,9,0
2340,2019-09-30 23:32:52+00:00,1178815001123065861,RT @opsoms: CONVOCATORIA DE ARTÍCULOS ❗️\n\n📘 ...,13,0
2341,2019-09-30 00:23:19+00:00,1178465311672143874,RT @biotecfarffucv: En #Biotecfar #SalvamosVid...,81,0
2342,2019-09-30 00:19:54+00:00,1178464452481605638,Cuántas tazas de café hay que tomar al día par...,0,0


#Procesamiento de data

In [ ]:
# dummy input (after cleaning)
o_df = extract_tweet_data(input_user)
test_df = o_df.copy()
test_df.text = test_df.text.map(lambda t: t.split())
test_df

Obtained 73.3% tweets in spanish from total of 3198 tweets from user @mferna.


,created_at,id,text,retweet_count,like_count
0,2022-05-03 01:37:08+00:00,1521302809450369026,"[Guía, de, evaluación, para, la, conservación,...",2,1
1,2022-04-27 15:56:31+00:00,1519344752981909504,"[RT, @gorka_orive:, Os, recomiendo, seguir, la...",115,0
2,2022-04-27 15:56:09+00:00,1519344657590800391,"[RT, @DrPontecarlosi:, Nuevas, guías, sobre, u...",74,0
3,2022-04-27 14:56:02+00:00,1519329529474555905,"[Sobre, la, Hepatitis, Infantil, Desconocida.....",0,0
4,2022-04-26 01:20:00+00:00,1518761781325451266,"[Adenovirus, 41, o, covid,, sospechosos, tras,...",1,0
...,...,...,...,...,...
2339,2019-10-02 14:51:45+00:00,1179408633756618754,"[RT, @campussanofi:, 👍👏, @AbbottGlobal, y, @sa...",9,0
2340,2019-09-30 23:32:52+00:00,1178815001123065861,"[RT, @opsoms:, CONVOCATORIA, DE, ARTÍCULOS, ❗️...",13,0
2341,2019-09-30 00:23:19+00:00,1178465311672143874,"[RT, @biotecfarffucv:, En, #Biotecfar, #Salvam...",81,0
2342,2019-09-30 00:19:54+00:00,1178464452481605638,"[Cuántas, tazas, de, café, hay, que, tomar, al...",0,0


In [ ]:
word = 'de'  # Test word

# Flatten text column
total_word_list = [word for word_list in o_df.text.tolist() for word in word_list]

word_count = total_word_list.count(word)  # 0TH VARIABLE
total_word_count = len(total_word_list)
freq = word_count/total_word_count  # FIRST VARIABLE


o_df['word_count'] = o_df.text.map(lambda t: t.count(word))
tweets_containing = o_df.word_count.astype(bool).sum()  #LIKE_COUNT_FLAG
like_count = o_df.like_count.sum()  # ARREGLAR
like_rate = like_count/tweets_containing  # SECOND


retweet_count = o_df.retweet_count.sum()  #ARREGLAR
retweet_rate = retweet_count/tweets_containing  # THIRD

# 4 and 5
popularity  = retweet_rate/freq
polemicity  = retweet_rate/like_rate

#BUSCAR POR BIGRAMAS Y TRIGRAMAS

print(f'\'{word}\' found {word_count} times.')
print(f'freq: {freq}.')
print(f'like_rate: {like_rate}.')
print(f'retweet_rate: {retweet_rate}.')
print(f'popularity: {popularity}.')
print(f'polemicity: {polemicity}.')
o_df

'de' found 0 times.
freq: 0.0.
like_rate: 0.9640579710144928.
retweet_rate: 261.46608695652174.
popularity: inf.
polemicity: 271.2140709561034.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in double_scalars


,created_at,id,text,retweet_count,like_count,word_count
0,2022-05-03 01:37:08+00:00,1521302809450369026,Guía de evaluación para la conservación de dat...,2,1,3
1,2022-04-27 15:56:31+00:00,1519344752981909504,RT @gorka_orive: Os recomiendo seguir la infor...,115,0,3
2,2022-04-27 15:56:09+00:00,1519344657590800391,RT @DrPontecarlosi: Nuevas guías sobre uso de ...,74,0,2
3,2022-04-27 14:56:02+00:00,1519329529474555905,Sobre la Hepatitis Infantil Desconocida... htt...,0,0,0
4,2022-04-26 01:20:00+00:00,1518761781325451266,"Adenovirus 41 o covid, sospechosos tras las he...",1,0,1
...,...,...,...,...,...,...
2339,2019-10-02 14:51:45+00:00,1179408633756618754,RT @campussanofi: 👍👏 @AbbottGlobal y @sanofi s...,9,0,4
2340,2019-09-30 23:32:52+00:00,1178815001123065861,RT @opsoms: CONVOCATORIA DE ARTÍCULOS ❗️\n\n📘 ...,13,0,1
2341,2019-09-30 00:23:19+00:00,1178465311672143874,RT @biotecfarffucv: En #Biotecfar #SalvamosVid...,81,0,1
2342,2019-09-30 00:19:54+00:00,1178464452481605638,Cuántas tazas de café hay que tomar al día par...,0,0,1


In [ ]:
def variables(word, df):
    # Flatten text column
    total_word_list = [word for word_list in df.text.tolist() for word in word_list]

    word_count = total_word_list.count(word)
    total_word_count = len(total_word_list)
    freq = word_count/total_word_count

    df['word_count'] = df.text.map(lambda t: t.count(word))
    tweets_containing = df['word_count'].astype(bool).sum()
    df.drop(['word_count'], axis=1)
    like_count = df.like_count.sum()
    like_rate = like_count/tweets_containing

    retweet_count = df.retweet_count.sum()
    retweet_rate = retweet_count/tweets_containing

    popularity  = retweet_rate/freq
    polemicity  = retweet_rate/like_rate

    variables = (freq, like_rate, retweet_rate, popularity, polemicity)
    return variables, word_count

In [ ]:
variables('de', test_df)

((0.053539296293220386,
  1.3125493291239148,
  355.9818468823994,
  6648.98255167162,
  271.21407095610346),
 1934)